In [16]:
from __future__ import division
import sys

import numpy as np
import networkx as nx
import sklearn.metrics as met
from sklearn.metrics.pairwise import euclidean_distances

import matplotlib.pyplot as plt

from itertools import repeat

from Queue import Queue
from threading import Thread
from threading import current_thread

MIN_EXPANSION_SIZE = 10
MAX_DELETED_NEURONS = 3

##########################################################################################################################

#function to generate real valued som for graph input
def initialise_network(ID, X):
    
    #network will be a one dimensional list
    network = nx.Graph(ID = ID)
    
    #initialise a network with just one neuron
    network.add_node(1)
    
    #id of node
    network.node[1]["ID"] = "{}-01".format(ID)
    
    #assign a random vector in X to be the weight
    V = np.expand_dims(X[np.random.randint(len(X))])
    
    #return network
    return network, V

#########################################################################################################################

def precompute_sigmas(sigma, num_epochs):
    
    return np.array([sigma * np.exp(-np.array([2]) * sigma * e / np.array([num_epochs])) 
                     for e in np.array(range(num_epochs))])

##########################################################################################################################
##TODO
# function to train SOM on given graph
def train_network(X, network, V, num_epochs, eta_0, pre_computed_sigmas):
    
    #initial learning rate
    eta = eta_0
    
    #list if all patterns to visit
    training_patterns = range(len(X))
    
    #shortest path matrix
    shortest_path = nx.floyd_warshall_numpy(network)
    
    for e in range(num_epochs):
        
        #shuffle nodes
        np.random.shuffle(training_patterns)
        
        # iterate through N nodes of graph
        for i in training_patterns:
            
            #data point to consider
            x = X[i]
            
            #determine winning neuron
            closest_neuron = winning_neuron(x, V)
            
            # update weights
            V = update_weights(x, V, closest_neuron, shortest_path, eta, pre_computed_sigmas[e])
            
    return V
        
##########################################################################################################################

# winning neuron
def winning_neuron(x, V):
    
    distances = np.linalg.norm(x - V, axis=1)
    
    return distances.argmin()

##########################################################################################################################

# function to update weights
def update_weights(x, V, winning_neuron, eta, sigma):
    
    #weight update (vectorised)
    V += np.dot(np.diag(eta * np.exp(- shortest_path_length[winning_neuron] ** 2 / (np.array([2]) * sigma ** 2))), 
                      (x - V))
    
    return V

########################################################################################################################   

# assign nodes into clusters
def assign_nodes(names, X, network, V):
    
    #distance from each datapoint (row) to each weight vector (column)
    distances = euclidean_distances(X, V)
    
    #minium distance for each datapoint
    min_distances = np.min(distances, axis=1)
    
    #index of column giving minimum distance
    arg_min_distances = np.argmin(distances, axis=1)
    
    #nodes corresponding to minimum index (of length len(X))
    minimum_nodes = np.array([network.nodes()[n] for n in arg_min_distances])
    
    #list of neurons with no assignments
    empty_neurons = np.array([n for n in network.nodes() if n not in minimum_nodes])
    
    if empty_neurons.size > 0:
    
        ################################################DELETION####################################################

        #neighbours of deleted neurons
        neighbour_lists = np.array([network.neighbors(n) for n in empty_neurons])

        print "DELETING: {}".format(empty_neurons)
        
        #remove the nodes
        network.remove_nodes_from(empty_neurons)
        
        ##remove from V
        V = V[i in arg_min_distances for i in range(len(V))]
        
        #compute distances between all neurons in input space
        computed_neuron_distances = compute_euclidean_distances(network, V)
        
        ##connect separated components
        for neighbour_list in neighbour_lists:
            connect_components(network, neighbour_list, computed_neuron_distances)

        ############################################################################################################

    #array of errors
    errors = np.array([np.mean(min_distances[minimum_nodes == n]) for n in network.nodes()])
    
    #compute MQE
    MQE = np.mean(errors)
    
    print "MQE={}".format(MQE)
    
    ##array of assignments
    assignments = np.array([np.array([names[i] for i in np.where(minimum_nodes == n)[0]]) for n in network.nodes()])
    
    #zip zith nodes
    errors = {n: e for n, e in zip(network.nodes(), errors)}
    assignments = {n: a for n, a in zip(network.nodes(), assignments)}
    
    nx.set_node_attributes(network, "e", errors)
    nx.set_node_attributes(network, "ls", assignments)
    
    return MQE, empty_neurons.size, V

##########################################################################################################################

def compute_euclidean_distances(network, V):
    
    distances = euclidean_distances(V)
    
    return {network.nodes()[i] : {network.nodes()[j] : distances[i, j] for j in range(len(distances[i]))}
           for i in range(len(distances))}
    
######################################################################################################################### 


def connect_components(network, neighbour_list):
    
    print "NODES DELETED CONNECTING COMPONENTS"
    
    sub_network = network.subgraph(neighbour_list)
    
    connected_components = [sub_network.subgraph(c) for c in nx.connected_components(sub_network)]
    number_of_connected_components = len(connected_components)
    
    for i in range(number_of_connected_components):
        
        connected_component_1 = connected_components[i].nodes()
        
        for j in range(i + 1, number_of_connected_components):
            
            connected_component_2 = connected_components[j].nodes()
            
            distances = np.array([[computed_neuron_distances[n1][n2] for n2 in connected_component_2]
                                 for n1 in connected_component_1])
            
            min_n1, min_n2 = np.unravel_index(distances.argmin(), distances.shape)
            
            network.add_edge(connected_component_1[min_n1], 
                            connected_component_2[min_n2])
            
            print "CONNECTED NEURONS: {} and {}".format(connected_component_1[min_n1], 
                            connected_component_2[min_n2])

##########################################################################################################################
            
##function to identify neuron with greatest error
def identify_error_unit(network):
    
    errors = nx.get_node_attributes(network, "e")
    
    return max(errors, key=errors.get)

##########################################################################################################################

def expand_network(ID, named_X, network, V, computed_neuron_distances, error_unit):
        
    #identify neighbour pointing closet
    error_unit_neighbours = network.subgraph(network.neighbors(error_unit))
    
    #id of new node
    id = max(network) + 1
    
    #add new node to map
    network.add_node(id)
    
    ##id
    network.node[id]["ID"] = "{}-{}".format(ID, str(id).zfill(2))
    
    #v goes to random vector in range of error unit
    ls = network.node[error_unit]["ls"]
    
    print "EXPANDING NETWORK"
    print "ERROR={}".format(network.node[error_unit]["e"])
    print "LS={}".format(ls)
    
    r = np.random.randint(len(ls))
    v = named_X[ls[r]]
    
    #add edges to map
    
    #connect error unit and new node
    network.add_edge(error_unit, id)
    
    if len(error_unit_neighbours) > 0:
        
        ##find closest neighbour
        distances = np.array([computed_neuron_distances[error_unit][n] for n in error_unit_neighbours])
        closest_neighbour = min(distances, key=distances.get)
        
        #connect to error unit and closest neighbour
        network.add_edge(closest_neighbour, id)
        
    #add v to V
    np.vstack([V, v])    
    
    return V
        
##########################################################################################################################
##########################################################################################################################

##GHSOM algorithm
def ghsom(ID, named_X, lam, eta, sigma, e_0, e_sg, e_en, q):
    
    #separate names and matrix of node embedding
    names, X = zip(*named_X.items())
    names = np.array(names)
    X = np.array(X)
    
    #create som for this neuron
    network = initialise_network(ID, X)
    
    #train for lamda epochs
    train_network(X, network, lam, eta, sigma)
    
    #classify nodes and compute error
    MQE, num_deleted_neurons = assign_nodes(names, X, network)
    
    ##som growth phase
    #repeat until error is low enough
    while MQE > e_sg * e_0 and num_deleted_neurons < MAX_DELETED_NEURONS:
        
        #find neuron with greatest error
        error_unit = identify_error_unit(network)
        
        #expand network
        expand_network(ID, named_X, network, error_unit)
        
        #train for lam epochs
        train_network(X, network, lam, eta, sigma)

        #calculate mean network error
        MQE, deleted_neurons = assign_nodes(names, X, network)
        num_deleted_neurons += deleted_neurons
        
        
        
    ##neuron expansion phase
    #iterate thorugh all neruons and find neurons with error great enough to expand
    for i, d in network.nodes(data=True):
        
        #unpack
        node_id = d["ID"]
        ls = d["ls"]
        e = d["e"]
        
        #check error
        if (e > e_en * e_0 and len(ls) > MIN_EXPANSION_SIZE and num_deleted_neurons < MAX_DELETED_NEURONS):
            
            id = "{}-{}".format(ID, node_id) 
                
            sub_X = {k: named_X[k] for k in ls}
            
            print "submitted job: ID={}, e={}".format(id, e)
            
            #add these parameters to the queue
            q.put((id, sub_X, lam, eta, sigma, e, e_sg, e_en))
    
    #return network
    return network, MQE

##########################################################################################################################
##########################################################################################################################

def label_nodes(G, networks):
    
    for _, network, _ in networks: 
        
        for _, d in network.nodes(data=True):
            
            community = d["ID"]
            layer = community.count("-")
            assignment_string = "assigned_community_layer_{}".format(layer)
            
            for node in d["ls"]:
                
                G.node[node][assignment_string] = community


##########################################################################################################################

def NMI_one_layer(G, label, layer):
    
    #actual community for this layer
    actual_community_labels = np.array([v for k, v in nx.get_node_attributes(G, label).items()])
    
    #predicted communitiy for this layer
    predicted_community_labels = np.array([v for k, v in nx.get_node_attributes(G, 
                                                                                "assigned_community_layer_{}".format(layer))])

    return met.normalized_mutual_info_score(actual_community_labels, predicted_community_labels)

def NMI_all_layers(G, labels):
    
    return np.array([NMI_one_layer(G, labels[i], i) for i in range(len(labels))])

##########################################################################################################################

## get embedding TERRIBLE but staying
def get_embedding(G):
    
    #get number of niodes in the graph
    num_nodes = nx.number_of_nodes(G)
    
    #dimension of embedding
    dim = 0
    while 'embedding'+str(dim) in G.node[G.nodes()[0]]:
        dim += 1
    
    #initialise embedding
    X = np.array([[d["embedding{}".format(j)] for j in range(dim)] for n, d in G.nodes(data=True)])
    
    return X

##########################################################################################################################

def process_job(q, networks):
    
    #unpack first element of queue
    #contains all the para,eters for GHSOM
    ID, X, lam, eta, sigma, e_0, e_sg, e_en = q.get()

    #run GHSOM and return a network and MQE
    n, e = ghsom(ID, X, lam, eta, sigma, e_0, e_sg, e_en, q)

    #append result to networks list
    networks.append((ID, n, e))

    #mark task as done
    q.task_done()

def worker(q, networks):
    
    #continually poll queue for jobs 
    while True:
        process_job(q, networks)

def main(params, gml_filename, lam=10000, num_threads=1):
    
    #network
    G = nx.read_gml(gml_filename)
    
    #embedding matrix
    X = get_embedding(G)
    
    #zip with names
    named_X = {k: v for k, v in zip(G.nodes(), X)}
    
    ##list of returned networks
    networks = []
    
    #initilise worker queue
    q = Queue()
    
    ##initial MQE is variance of dataset
    m = np.mean(X, axis=0)
    MQE_0 = np.mean([np.linalg.norm(x - m) for x in X])
    
    #add initial layer of ghsom to queue
    q.put(("01", named_X, lam, params["eta"], params["sigma"], MQE_0, params["e_sg"], params["e_en"]))
    
    if num_threads > 1:
    
        #initialise threads
        for i in range(num_threads):

            t = Thread(target=worker, args=(q, networks))
            t.setDaemon(True)
            t.start()

        #finally wait until queue is empty and all tasks are done
        q.join()
        
    else :
        
        #single thread
        while not q.empty():
            process_job(q, networks)
    
    print "DONE"
    
    return G, networks

In [17]:
params = {'eta': np.array([0.0001]),
         'sigma': np.array([1]),
          'e_sg': np.array([0.8]),
         'e_en': np.array([0.8])}

In [18]:
# %%time 
%prun G, networks = main(params=params, gml_filename="embedded_yeast_uetz.gml", num_threads=1)

MQE=8.1720267696
ERROR UNIT
{1: 8.172026769596199}
1
EXPANDING NETWORK
ERROR=8.1720267696
LS=[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 22

In [ ]:
label_graph(G, networks)

In [ ]:
NMI_all_layersll_layers(G, labels=["communityfirstlevel", "communitysecondlevel"])

In [31]:
for id, network, e in networks:
    
    print id
    print network.graph["ID"]
    print nx.floyd_warshall_numpy(network)
    print 
    for n, d in network.nodes(data=True):
        print d["ID"]
#         print str(n + 1).zfill(2)
        print np.array([G.node[node]["label"] for node in d["ls"]])
        print

01
{}
[[ 0.  1.  1.]
 [ 1.  0.  1.]
 [ 1.  1.  0.]]



KeyError: 'ID'

In [15]:
%timeit np.array([3]) ** 2

The slowest run took 10.75 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.13 µs per loop


In [14]:
%timeit 3 ** 2

The slowest run took 242.04 times longer than the fastest. This could mean that an intermediate result is being cached.
100000000 loops, best of 3: 15.1 ns per loop


In [19]:
x = np.ones(10)

In [20]:
x

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [21]:
V = np.random.rand(4, 10)

In [109]:
V

array([[ 0.49529815,  0.18757437,  0.34372296,  0.94761752,  0.01004122,
         0.24339167,  0.76693577,  0.92906051,  0.29371815,  0.19445442],
       [ 0.41539352,  0.10635126,  0.13549128,  0.1208399 ,  0.56317103,
         0.56504311,  0.35461997,  0.75235715,  0.75311595,  0.90571066],
       [ 0.26849324,  0.13887693,  0.26918758,  0.53133791,  0.83181293,
         0.65800801,  0.19624571,  0.27947993,  0.90247327,  0.35431028],
       [ 0.00835532,  0.21813548,  0.96096738,  0.94383539,  0.56248537,
         0.4845722 ,  0.52816581,  0.65962402,  0.01317999,  0.78644797],
       [ 0.51383327,  0.65470872,  0.90030044,  0.74498116,  0.12355387,
         0.83561573,  0.47583729,  0.87859551,  0.05922463,  0.96739777],
       [ 0.69772866,  0.42101705,  0.52387138,  0.36964618,  0.79153711,
         0.70393338,  0.7006208 ,  0.84793905,  0.54686991,  0.33514087],
       [ 0.70820633,  0.85819298,  0.98531544,  0.34022415,  0.57112018,
         0.52750527,  0.9148991 ,  0.21071674

In [113]:
%%timeit 
euclidean_distances(V)

The slowest run took 29.78 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 120 µs per loop


In [25]:
%timeit d1 = np.array([np.linalg.norm(x - v) for v in V])

The slowest run took 5.79 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 29 µs per loop


In [24]:
d1

array([ 1.25720888,  1.57568538,  1.82639795,  1.40716874])

In [32]:
%timeit d2 = np.linalg.norm(x - V, axis=1)

The slowest run took 10.10 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 15.3 µs per loop


In [52]:
d2

array([ 1.25720888,  1.57568538,  1.82639795,  1.40716874])

In [54]:
d2.argmin()

0

In [42]:
%%timeit 
np.array([np.random.shuffle(range(100)) for i in range(200)])

The slowest run took 5.92 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 693 µs per loop


In [38]:
np.array([np.random.permutation(range(100)) for i in range(200)]).shape

(200, 100)

In [41]:
%%timeit
patterns = range(100)
for i in range(200):
    np.random.shuffle(patterns)

The slowest run took 4.26 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 543 µs per loop


In [40]:
%%timeit
patterns = range(100)
for i in range(200):
    np.random.permutation(patterns)

100 loops, best of 3: 3.98 ms per loop


In [43]:
import networkx as nx

In [44]:
G = nx.karate_club_graph()

In [45]:
nx.floyd_warshall_numpy(G)

matrix([[ 0.,  1.,  1., ...,  1.,  2.,  2.],
        [ 1.,  0.,  1., ...,  2.,  2.,  2.],
        [ 1.,  1.,  0., ...,  2.,  1.,  2.],
        ..., 
        [ 1.,  2.,  2., ...,  0.,  1.,  1.],
        [ 2.,  2.,  1., ...,  1.,  0.,  1.],
        [ 2.,  2.,  2., ...,  1.,  1.,  0.]])

In [51]:
for e in np.array(range(1)):
    print type(e)

<type 'numpy.int64'>


In [65]:
X = np.random.uniform(size=(100, 10))

In [66]:
X.shape

(100, 10)

In [114]:
V = np.random.uniform(size=(50, 10))

In [115]:
V.shape

(50, 10)

In [116]:
v = V[10]

In [119]:
%%timeit
np.linalg.norm(X - v, axis=1)

The slowest run took 9.28 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 18.5 µs per loop


In [120]:
%%timeit
euclidean_distances(X, V)[10]

The slowest run took 11.03 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 124 µs per loop


In [60]:
from sklearn.metrics.pairwise import euclidean_distances

In [72]:
v = np.random.rand(10)

In [93]:
%%timeit
d = euclidean_distances(X, np.expand_dims(v,axis=0))
np.squeeze(d)

The slowest run took 6.81 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 86.7 µs per loop


In [92]:
%%timeit
np.linalg.norm(X-v, axis=1)

The slowest run took 16.19 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 18.3 µs per loop


In [87]:
np.expand_dims(v,axis=0)

array([[ 0.86165423,  0.86393597,  0.54176829,  0.74506783,  0.47238384,
         0.04732205,  0.42682069,  0.22436778,  0.09487702,  0.14713109]])

array([ 1.12988925,  1.22482168,  1.38759476,  1.25844721,  1.20800464,
        1.61205317,  1.18898783,  1.57134352,  1.4357881 ,  1.29428861,
        0.76502481,  1.40508946,  1.3369364 ,  1.30399373,  1.33784348,
        1.61825203,  1.10802022,  1.53395079,  1.68038185,  1.53273757,
        1.48455036,  1.45308392,  1.53480268,  1.35857086,  1.03782755,
        1.26118556,  1.40668269,  1.72257661,  1.45673875,  1.20980696,
        1.15707178,  1.31310238,  1.21313389,  1.63797217,  1.4377278 ,
        1.46564403,  1.156674  ,  1.11945899,  1.32434497,  1.60576223,
        1.43649465,  1.12637451,  1.20218976,  1.01408283,  1.19630106,
        1.14655355,  1.34197424,  1.38412298,  1.54643675,  1.1775175 ,
        0.74262937,  1.26378427,  0.94512418,  1.31216168,  1.25016525,
        0.86835855,  0.92839379,  1.27398001,  1.04907224,  1.41949531,
        1.50928064,  1.60320921,  1.65079118,  1.22573845,  1.36655758,
        1.36783467,  1.41640277,  1.30323882,  0.92059426,  1.18

In [79]:
sp.min(d, axis=1).shape

AttributeError: 'module' object has no attribute 'min'

In [78]:
import scipy as sp

In [105]:
distances = np.random.uniform(size=(10, 10))

In [106]:
arg_min_distances = distances.argmin(axis=1)

In [107]:
arg_min_distances

array([0, 8, 6, 1, 2, 6, 6, 6, 7, 0])

In [108]:
[i in arg_min_distances for i in range(10)]

[True, True, True, False, False, False, True, True, True, False]

In [125]:
G = nx.karate_club_graph()

In [127]:
G.remove_node(0)

In [128]:
for i in range(10):
    print G.neighbors(5)

[16, 10, 6]
[16, 10, 6]
[16, 10, 6]
[16, 10, 6]
[16, 10, 6]
[16, 10, 6]
[16, 10, 6]
[16, 10, 6]
[16, 10, 6]
[16, 10, 6]


In [129]:
V = np.random.uniform(size=(10, 2))

In [135]:
%timeit np.vstack([V, np.random.rand(2)])

The slowest run took 30.59 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 7.88 µs per loop


In [136]:
d = np.random.rand(4,4)

In [137]:
d

array([[ 0.93972975,  0.65445338,  0.85797874,  0.00275317],
       [ 0.77358549,  0.36047962,  0.76187044,  0.9796929 ],
       [ 0.48033511,  0.74553867,  0.83923645,  0.83656042],
       [ 0.51520627,  0.01883697,  0.81623215,  0.69278958]])

In [140]:
keys = ["one", "two", "three", "four"]

In [141]:
{keys[i]: {keys[j]: d[i,j] for j in range(len(d[i]))} for i in range(len(d))}

{'four': {'four': 0.69278958245266331,
  'one': 0.51520626539052738,
  'three': 0.81623215235313107,
  'two': 0.018836967846983632},
 'one': {'four': 0.0027531722308948847,
  'one': 0.93972975362457611,
  'three': 0.85797873903584243,
  'two': 0.65445338026773647},
 'three': {'four': 0.83656041699512196,
  'one': 0.48033510776842603,
  'three': 0.83923645494436583,
  'two': 0.74553866626637189},
 'two': {'four': 0.97969290303959267,
  'one': 0.77358548736724631,
  'three': 0.76187043742294702,
  'two': 0.3604796239127902}}